# Project 4
## Students:
 > Austin Houston,
 > Alexander Krneta
 
 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

print(tf.__version__)# you may want to upgrade to 2.10.0 

2.9.0


## Task 1

In [2]:
# needs to be able to change number of heads?
# input to Transformer block is broken

In [3]:
class TransformerModel(keras.Model):
    def __init__(self, vocab_size, embed_dim=256, num_heads=2, num_blocks=1, ff_dim=256, maxlen=80, dropout_rate=0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.max_length = maxlen
        self.dropout_rate = dropout_rate
        self.num_blocks = num_blocks
        self.embeddings = None
        self.outputs = None

        self.inputs = keras.Input(shape=(None, self.embed_dim))


    def EmbeddingLayer(self):
        # Initialize embeddings
        self.token_embedding = layers.Embedding(input_dim=self.vocab_size, output_dim=self.embed_dim, input_length=self.max_length)
        self.positional_embedding = layers.Embedding(input_dim=self.max_length, output_dim=self.embed_dim, input_length=self.max_length, embeddings_initializer=keras.initializers.RandomUniform())
        self.dropout = layers.Dropout(self.dropout_rate)

        position_ids = tf.range(start=0, limit=tf.shape(self.inputs)[-1], delta=1, dtype=tf.int32)
        position_embedding = self.positional_embedding(position_ids)
        token_embedding = self.token_embedding(self.inputs)
        self.embeddings = token_embedding + position_embedding


    def TransformerBlock(self):
        # Multi-Head Attention layer 
        # Sums the input to the block and the output from the first dropout
        attention = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embed_dim)(self.embeddings, self.embeddings)
        attention = layers.Dropout(rate=self.dropout_rate)(attention)
        attention = layers.LayerNormalization(epsilon=1e-6)(layers.Add()([self.embeddings, attention]))
        
        # Feed-Forward Dense layer
        # Sums the output of the first LayerNormalization and second dropout
        dense = layers.Dense(units=self.ff_dim, activation='relu')(attention)
        dense = layers.Dropout(rate=self.dropout_rate)(dense)
        dense = layers.Dense(units=self.embed_dim)(dense)
        dense = layers.Dropout(rate=self.dropout_rate)(dense)
        dense = layers.LayerNormalization(epsilon=1e-6)(layers.Add()([attention, dense]))

        self.outputs = layers.Dense(units=self.embed_dim)(dense)

    def create_model(self,vocab_size, embed_dim, num_heads, num_blocks, ff_dim, maxlen, dropout_rate):
        
        self.EmbeddingLayer()
        self.TransformerBlock()

        model = tf.keras.models.Model(inputs = self.inputs, outputs=self.outputs)

        # Compile the model with sparse categorical crossentropy loss and Adam optimizer
        model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy']
        )
        return model

In [4]:
model = TransformerModel(vocab_size = 10)
model = model.create_model(vocab_size = 100, embed_dim=256, num_heads=2, num_blocks=1, ff_dim=256, maxlen=80, dropout_rate=0.1)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-01 16:26:16.131700: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-01 16:26:16.132790: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [40]:
emb = model.EmbeddingLayer()

In [42]:
emb.output_shape

(None, None, 256, 256)

## Task 2

In [1]:
# needs work with special characters

In [51]:
class Dataset():
    def __init__(self, filepath):
        with open(filepath, 'r') as f:
            self.text = f.read()
        self.vocab = None
        self.reverse_vocab = None

    def prep_text(self):
        self.text = self.text.lower()
        self.text = ''.join([c for c in self.text if c.isalnum() or c.isspace()])
    
    def tokenize_text(self):
        words = np.unique(self.text.split())
        self.vocab = {w: i+1 for i, w in enumerate(words)}
        self.reverse_vocab = {i+1: w for i, w in enumerate(words)}
        self.text = [self.vocab[w] for w in self.text.split()]
    
    def create_dataset(self):
        self.prep_text()
        self.tokenize_text()
        x = np.array(self.text[:-1])
        y = np.array(self.text[1:])
        x = to_categorical(x, num_classes=len(self.vocab)+1)
        return x, y, self.vocab


In [52]:
data = Dataset('beatles.txt')
x, y, vocab = data.create_dataset()

In [57]:
vocab

{'0': 1,
 '1': 2,
 '1234': 3,
 '15': 4,
 '17': 5,
 '2': 6,
 '3': 7,
 '4': 8,
 '50': 9,
 '56789': 10,
 '9': 11,
 '909': 12,
 'a': 13,
 'aaaaahhhhhhhhhh': 14,
 'aaaah': 15,
 'aaahhh': 16,
 'able': 17,
 'about': 18,
 'above': 19,
 'abrigado': 20,
 'accidents': 21,
 'aches': 22,
 'acorns': 23,
 'acquainted': 24,
 'across': 25,
 'act': 26,
 'acts': 27,
 'admit': 28,
 'advice': 29,
 'affection': 30,
 'afraid': 31,
 'after': 32,
 'afternoon': 33,
 'again': 34,
 'against': 35,
 'age': 36,
 'ago': 37,
 'agree': 38,
 'ah': 39,
 'ahead': 40,
 'aint': 41,
 'air': 42,
 'al': 43,
 'albert': 44,
 'alerted': 45,
 'all': 46,
 'allan': 47,
 'allein': 48,
 'alley': 49,
 'almost': 50,
 'alone': 51,
 'along': 52,
 'aloud': 53,
 'already': 54,
 'alright': 55,
 'although': 56,
 'always': 57,
 'am': 58,
 'american': 59,
 'amore': 60,
 'amoving': 61,
 'amsterdam': 62,
 'an': 63,
 'and': 64,
 'andarollin': 65,
 'andern': 66,
 'angel': 67,
 'angry': 68,
 'anna': 69,
 'annoyed': 70,
 'another': 71,
 'answer': 72,

## Task 3

In [ ]:
class GenerateText():
    def __init__(self, model, vocab):
        pass

    
    def generate_text(self, start_string, num_generate=100):
        #generate text using the model and vocab, start with the start_string and generate num_generate words
        pass

## Task 4: Model Traning and Testing

In [ ]:
#Train the model while periodically generating text to show progress
def train_model(model, vocab, x, y, epochs=50):
    return model


# Report

## Introduction

## Results

## Conclusion

## How to Run Code

Please include any special libraries and list your tf version here.